In [1]:
%pwd

'D:\\android_gan_mal2020'

In [2]:
import numpy as np
import pandas as pd
import pickle
import csv
import os
import torch
from torch_geometric.data import Data
from tqdm import tqdm
import networkx as nx
from glob import glob
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
import torch
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm
from scipy import sparse
from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv, SAGEConv, SGConv
import matplotlib.pyplot as plt
from scipy import sparse
np.random.seed(42)
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,accuracy_score
from torch_geometric.nn import VGAE
from torch import nn
import torch_geometric
from torch_geometric.data import DataLoader
from sklearn import metrics
import random

In [3]:
def get_graph(mat):
#     mat = sparse.load_npz(path)
#     print(mat.shape)
    mat_coo = mat.tocoo()
    mat_csr = mat_coo.tocsr()
    G_new = nx.Graph(mat_csr)
    return G_new
    

In [4]:
def load_feature(path):
    with open(path, 'rb') as f:
        total_feature = pickle.load(f)
        return total_feature

In [5]:
total_feature= load_feature(r'code_total_feature')

In [6]:
print(len(total_feature))

1000


In [7]:
class GraphSage(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GraphSage, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = SAGEConv(5, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.conv4 = SAGEConv(hidden_channels, hidden_channels)
        self.conv5 = SAGEConv(hidden_channels, hidden_channels)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels)
        self.lin3 = Linear(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)
        x = x.relu()
        x = self.conv5(x, edge_index)
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        # 3. Apply a final classifier
        x = F.dropout(x, training=self.training)
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.lin3(x)   
        x = self.lin(x)
        return x

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [9]:
black_box_model = GraphSage(hidden_channels=32)
optimizer = torch.optim.Adam(black_box_model.parameters(), lr=0.004)
criterion = torch.nn.CrossEntropyLoss()
black_box_model.load_state_dict(torch.load(r"GraphSagemal2020full_94acc.pt"))
black_box_model = black_box_model.to(device)

In [10]:
black_box_model

GraphSage(
  (conv1): SAGEConv(5, 32)
  (conv2): SAGEConv(32, 32)
  (conv3): SAGEConv(32, 32)
  (conv4): SAGEConv(32, 32)
  (conv5): SAGEConv(32, 32)
  (lin1): Linear(in_features=32, out_features=32, bias=True)
  (lin2): Linear(in_features=32, out_features=32, bias=True)
  (lin3): Linear(in_features=32, out_features=32, bias=True)
  (lin): Linear(in_features=32, out_features=2, bias=True)
)

In [11]:
class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = SAGEConv(in_channels, out_channels)
        self.conv5 = SAGEConv(out_channels, out_channels)
        self.conv_mu = SAGEConv(out_channels, out_channels)
        self.conv_logstd = SAGEConv(out_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv5(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [12]:
vae_generator = VGAE(VariationalGCNEncoder(5,32))
optimizer_vae_generator = torch.optim.Adam(vae_generator.parameters(), lr=0.01)
vae_generator.load_state_dict(torch.load(r"vgae_mal_final_no_diag.pt"))
vae_generator = vae_generator.to(device)

vgae


In [13]:
class GraphSageDis(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GraphSageDis, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = SAGEConv(5, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.conv4 = SAGEConv(hidden_channels, hidden_channels)
        self.conv5 = SAGEConv(hidden_channels, hidden_channels)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels)
        self.lin3 = Linear(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)
        x = x.relu()
        x = self.conv5(x, edge_index)
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        # 3. Apply a final classifier
        x = F.dropout(x, training=self.training)
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.lin3(x)   
        x = self.lin(x)
        return x

In [14]:
discriminator_model = GraphSageDis(hidden_channels=32)
optimizer_discriminator = torch.optim.Adam(discriminator_model.parameters(), lr=0.01)
criterion_discriminator = torch.nn.CrossEntropyLoss()
discriminator_model = discriminator_model.to(device)

In [15]:
class MyEnsemble(torch.nn.Module):
    def __init__(self, vae_generator, discriminator):
        super(MyEnsemble, self).__init__()
        self.vae_generator = vae_generator
        self.discriminator = discriminator
        
    def forward(self, x, edge_index,batch,full):
        z = self.vae_generator.encode(x,edge_index)
#         print(z)
        adj=torch.exp(-torch.cdist(z,z, p=2.0))
        adj = (adj>0.98).float()*1
#         print(adj)
        #to remove self loop
        generated_edge_index_sparse=torch_geometric.utils.dense_to_sparse(adj)[0]    
         # self loop remove
        generated_edge_index_sparse=torch_geometric.utils.remove_self_loops(generated_edge_index_sparse)[0]
        try:
        # convert back to adjacency matrix to perform and and or operation
            generated_edge_index_dense=torch_geometric.utils.to_dense_adj(generated_edge_index_sparse,max_num_nodes=1000)[0]
        except:
            generated_edge_index_dense=torch_geometric.utils.to_dense_adj(edge_index,max_num_nodes=1000)[0]
        #perform and operation with full graph
        generated_edge_index_valid_links=torch.logical_and(full,generated_edge_index_dense)*1
        #perform or operation with the original edge_index to preserver the malware characteristic
        edge_index_dense=torch_geometric.utils.to_dense_adj(edge_index,max_num_nodes=1000)[0]
        generated_edge_index_final=torch.logical_or(generated_edge_index_valid_links,edge_index_dense)*1
        #cover it into sparse to pass into discriminator
        generated_edge_index_final=torch_geometric.utils.dense_to_sparse(generated_edge_index_final)[0]
#         print("yupp")
#         print(generated_edge_index_final)
        output = self.discriminator(x,generated_edge_index_final,batch)
        return output,z

In [16]:
generator=MyEnsemble(vae_generator, discriminator_model)
generator=generator.to(device)
print(generator)

MyEnsemble(
  (vae_generator): VGAE(
    (encoder): VariationalGCNEncoder(
      (conv1): SAGEConv(5, 32)
      (conv5): SAGEConv(32, 32)
      (conv_mu): SAGEConv(32, 32)
      (conv_logstd): SAGEConv(32, 32)
    )
    (decoder): InnerProductDecoder()
  )
  (discriminator): GraphSageDis(
    (conv1): SAGEConv(5, 32)
    (conv2): SAGEConv(32, 32)
    (conv3): SAGEConv(32, 32)
    (conv4): SAGEConv(32, 32)
    (conv5): SAGEConv(32, 32)
    (lin1): Linear(in_features=32, out_features=32, bias=True)
    (lin2): Linear(in_features=32, out_features=32, bias=True)
    (lin3): Linear(in_features=32, out_features=32, bias=True)
    (lin): Linear(in_features=32, out_features=2, bias=True)
  )
)


In [17]:
# overall_graph_path=r'code_block_id_tr_reduced_tr.npz'
# sparse_overall_graph = sparse.load_npz(overall_graph_path)
# overall_graph_edge_index_coo=sparse_overall_graph.tocoo()
# overall_graph_edge_index = torch.tensor([overall_graph_edge_index_coo.row.tolist(),overall_graph_edge_index_coo.col.tolist()], dtype=torch.long) 
# pos_overall_graph_edge_index = overall_graph_edge_index.to(device)
# pos_overall_graph_edge_index =torch_geometric.utils.remove_self_loops(pos_overall_graph_edge_index)[0]
# pos_overall_adj=torch_geometric.utils.to_dense_adj(pos_overall_graph_edge_index)[0]
# print(pos_overall_adj)

In [18]:
overall_graph=load_feature("overall_edge_list")
overall_graph_edge_index = torch.tensor([overall_graph[0],overall_graph[1]], dtype=torch.long) 
pos_overall_graph_edge_index = overall_graph_edge_index.to(device)
pos_overall_graph_edge_index = torch_geometric.utils.remove_self_loops(pos_overall_graph_edge_index)[0]
pos_overall_adj=torch_geometric.utils.to_dense_adj(pos_overall_graph_edge_index)[0]
print(pos_overall_adj)

tensor([[0., 1., 1.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 1., 0.]], device='cuda:0')


In [19]:
def mydecoder(edge_index,z,pos_overall_adj):
    adj=torch.exp(-torch.cdist(z,z, p=2.0))
    adj = (adj>0.98).float()*1
#     adj = torch.matmul(z, z.t())
#     adj=torch.sigmoid(adj)
    #to remove self loop
    generated_edge_index_sparse=torch_geometric.utils.dense_to_sparse(adj)[0] 
    
     # self loop remove
    generated_edge_index_sparse=torch_geometric.utils.remove_self_loops(generated_edge_index_sparse)[0]
    # convert back to adjacency matrix to perform and and or operation
    try:
        generated_edge_index_dense=torch_geometric.utils.to_dense_adj(generated_edge_index_sparse,max_num_nodes=1000)[0]
        exception=False
    except:
        generated_edge_index_dense=torch_geometric.utils.to_dense_adj(edge_index,max_num_nodes=1000)[0]
        exception=True
#     print(pos_overall_adj)
    #perform and operation with full graph
    generated_edge_index_valid_links=torch.logical_and(pos_overall_adj,generated_edge_index_dense)*1
    #print(generated_edge_index_valid_links)
    #perform or operation with the original edge_index to preserver the malware characteristic
    edge_index_dense=torch_geometric.utils.to_dense_adj(edge_index,max_num_nodes=1000)[0]
    generated_edge_index_final=torch.logical_or(generated_edge_index_valid_links,edge_index_dense)*1
    #cover it into sparse to pass into discriminator
    generated_edge_index_final=torch_geometric.utils.dense_to_sparse(generated_edge_index_final)[0]
    return generated_edge_index_final,exception
    

In [20]:
#loading dataset malware
class YooChooseDatasetMal(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(YooChooseDatasetMal, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []
    @property
    def processed_file_names(self):
        return [r'code_block.dataset']

    def download(self):
        pass
        
    def process(self):
        data_list = []
        edge_index_path=[r"class1"]
        current=0
        for i in edge_index_path: 
            edge_indexes = glob(f'{i}/*.npz')
            print(len(edge_indexes))
            for edge_index_path in edge_indexes:
                current=current+1
                print(current)
                y=[1]
                y = torch.tensor(y)
                try:
                    sparse_arr = sparse.load_npz(edge_index_path)
                    sparse_arr.setdiag(0)
                    sparse_arr.eliminate_zeros()
                    edge_index_processed=sparse_arr
                    edge_index = torch.tensor([edge_index_processed.row.tolist(),edge_index_processed.col.tolist()], dtype=torch.long)
                    x = torch.tensor(total_feature,dtype=torch.float)
                    data = Data(x=x, edge_index=edge_index, y=y)
                    data_list.append(data)
                except Exception as e:
                    print(e)
                    print(edge_index_path)
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [21]:
custom_dataset= YooChooseDatasetMal(root=r'onlygraphclass1')

In [22]:
torch.manual_seed(12345)
dataset_malware = custom_dataset.shuffle()

train_dataset_malware = dataset_malware[:8474]
test_dataset_malware = dataset_malware[8474:]
print(test_dataset_malware)
print(f'Number of training graphs: {len(train_dataset_malware)}')
print(f'Number of test graphs: {len(test_dataset_malware)}')
print(dataset_malware.num_node_features)

YooChooseDatasetMal(3633)
Number of training graphs: 8474
Number of test graphs: 3633
5


In [23]:
#loading dataset malware
class YooChooseDatasetBen(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(YooChooseDatasetBen, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []
    @property
    def processed_file_names(self):
        return [r'code_block.dataset']

    def download(self):
        pass
        
    def process(self):
        data_list = []
        edge_index_path=[r"class0"]
        current=0
        for i in edge_index_path: 
            edge_indexes = glob(f'{i}/*.npz')
            print(len(edge_indexes))
            for edge_index_path in edge_indexes:
                current=current+1
                print(current)
                y=[0]
                y = torch.tensor(y)
                try:
                    sparse_arr = sparse.load_npz(edge_index_path)
                    sparse_arr.setdiag(0)
                    sparse_arr.eliminate_zeros()
                    edge_index_processed=sparse_arr
                    edge_index = torch.tensor([edge_index_processed.row.tolist(),edge_index_processed.col.tolist()], dtype=torch.long)
                    x = torch.tensor(total_feature,dtype=torch.float)
                    data = Data(x=x, edge_index=edge_index, y=y)
                    data_list.append(data)
                except Exception as e:
                    print(e)
                    print(edge_index_path)
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [24]:
custom_dataset_benign= YooChooseDatasetBen(root=r'onlygraphclass0')

In [25]:
torch.manual_seed(12345)
dataset_benign = custom_dataset_benign.shuffle()
train_dataset_benign = dataset_benign[:2586]
test_dataset_benign = dataset_benign[2586:]
print(test_dataset_benign)
print(f'Number of training graphs: {len(train_dataset_benign)}')
print(f'Number of test graphs: {len(test_dataset_benign)}')
print(dataset_benign.num_node_features)

YooChooseDatasetBen(1109)
Number of training graphs: 2586
Number of test graphs: 1109
5


In [26]:
def discriminator_train(loader):
    discriminator_model.train()
    running_loss = 0.0
    correct = 0
    benign_count=0
    malware_count=0
    for data in loader:  # Iterate in batches over the training dataset.
         data.x = data.x.to(device)
         data.edge_index= data.edge_index.to(device)
         data.batch=data.batch.to(device)
         data.y = data.y.to(device)
         out = discriminator_model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         pred = out.argmax(dim=1)    
         output=pred.cpu().numpy()
         for i in output:
            if i ==0:
                benign_count+=1
            else:
                malware_count+=1
         
         correct += int((pred == data.y).sum())
         loss = criterion_discriminator(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer_discriminator.step()  # Update parameters based on gradients.
         optimizer_discriminator.zero_grad() # Clear gradients.
         running_loss += loss.item() * data.num_graphs
    running_loss /= len(loader.dataset)
    print(benign_count)
    print(malware_count)
    return correct / len(loader.dataset),running_loss

In [27]:
def get_discriminator_prediction(data):
    discriminator_model.eval()
    data=data.to(device)
    out= discriminator_model(data.x, data.edge_index, data.batch)  
    pred = out.argmax(dim=1)
    pred_numpy = pred.cpu().numpy()
    return pred_numpy
    

In [28]:
def get_black_box_prediction(data):
    black_box_model.eval()
    data=data.to(device)
    out= black_box_model(data.x, data.edge_index, data.batch)  
    pred = out.argmax(dim=1)
#     print("yooo")
    return pred

In [29]:
benign_list=[]
for data in train_dataset_benign:
    data.y= torch.tensor([0])
    benign_list.append(data)

In [30]:
def true_positive_rate_calculator(prediction_list,true_list):
    confusion=confusion_matrix(true_list,prediction_list)
    print(confusion)
    TP=confusion[1][1]
    FN=confusion[1][0]
    true_positive_rate=TP/(TP+FN)
    return true_positive_rate
    
    

In [32]:
benign_train_loader = DataLoader(benign_list, batch_size=64, shuffle=True)
train_acc_benign,train_loss_benign = discriminator_train(benign_train_loader)

2586
0


In [34]:
#for testing not needed
count=0
true_positive_calcu=[]
#for batching will use data loader to batch and pass on to discriminator
for epoch in range(100):
    print("current iteration",epoch)
    generated_malware_list=[]
    exception_count=0
    generated_prediction=[]
    for malware in train_dataset_malware:
        
        try:
            random_data = random.choice(benign_list)
            edge_index_0=random_data.edge_index[0].split(int(len(random_data.edge_index[0])/2))[1]
            edge_index_1=random_data.edge_index[1].split(int(len(random_data.edge_index[1])/2))[1]
            benign_noise = torch.empty([2,int(len(random_data.edge_index[1])/2)],dtype=int)
            benign_noise[0]=edge_index_0
            benign_noise[1]=edge_index_1
            noise_edge_index=torch.cat((malware.edge_index,benign_noise),1)
            malware.edge_index=noise_edge_index
            count=count+1
            malware=malware.to(device)
            z = vae_generator.encode(malware.x,malware.edge_index)
            edge_index,exception=mydecoder(malware.edge_index,z,pos_overall_adj)
            if exception:
                exception_count+=1
            x = torch.tensor(total_feature,dtype=torch.float)
            data = Data(x=x, edge_index=edge_index)
            data.batch=torch.zeros(1000, dtype=torch.long)
            #get the black_box prediction
            out = get_black_box_prediction(data).cpu()
            generated_prediction.append(out)
            data.edge_index=data.edge_index.cpu()
            data.edge_index.cpu()
            data.x=data.x.cpu()
            data.batch=data.batch.cpu()
            data.y=out.cpu()
            out = out.cpu().numpy()
            data.batch=None
            generated_malware_list.append(data)
        except Exception as e:
            print(e)
    print("malware generation complete")
    print("graph generation",exception_count)
    pred_list_list=[l.tolist() for l in generated_prediction]
    predlist = [item for sublist in pred_list_list for item in sublist]
    # print(predlist)
    predlist.append(0)
    predlist.append(1)
    true_label_check=[1]*len(predlist)
    print("true positive rate after generation",true_positive_rate_calculator(predlist,true_label_check))
    tpr=true_positive_rate_calculator(predlist,true_label_check)
    if tpr<=0.20:
        print("saved here 20")
        torch.save(vae_generator.state_dict(), r"onlygraphvgaebb_final_"+str(tpr)+".pt")
    #combine_list=generated_malware_list+benign_list
    benign_train_loader = DataLoader(benign_list, batch_size=64, shuffle=True)
    malware_train_loader= DataLoader(generated_malware_list, batch_size=64, shuffle=True)
    train_acc_benign,train_loss_benign = discriminator_train(benign_train_loader)
    print("training benign discriminator")
    print(f'Epoch: {epoch:03d}, discriminator Train Acc: {train_acc_benign:.4f},discriminator Train loss:{train_loss_benign:.4f}')
    for inner_epoch in range(10):
        train_acc_malware,train_loss_malware = discriminator_train(malware_train_loader)
        print("training malware discriminator")
        print(f'Epoch: {inner_epoch:03d}, discriminator Train Acc: {train_acc_malware:.4f},discriminator Train loss:{train_loss_malware:.4f}')
    # train generator
    for generator_epoch in range(3):
        running_loss = 0.0
        running_loss_dis=0.0
        for data in train_dataset_malware:
            data.y=torch.tensor([0])
            data.batch=torch.zeros(1000, dtype=torch.long)
            random_data = random.choice(benign_list)
            edge_index_0=random_data.edge_index[0].split(int(len(random_data.edge_index[0])/2))[1]
            edge_index_1=random_data.edge_index[1].split(int(len(random_data.edge_index[1])/2))[1]
            benign_noise = torch.empty([2,int(len(random_data.edge_index[1])/2)],dtype=int)
            benign_noise[0]=edge_index_0
            benign_noise[1]=edge_index_1
            noise_edge_index=torch.cat((data.edge_index,benign_noise),1)
            data.edge_index=noise_edge_index
            data=data.to(device)
            out,z=generator(data.x,data.edge_index,data.batch,pos_overall_adj)
#             print("this is the output",out)
            loss = vae_generator.recon_loss(z,data.edge_index)
            loss_discriminator = criterion_discriminator(out, data.y)
    #        loss=loss_discriminator
    #         print(loss_discriminator)
            loss = loss+(1 / data.num_nodes) * vae_generator.kl_loss()+loss_discriminator
            loss.backward()  # Derive gradients.
            optimizer_vae_generator.step()  # Update parameters based on gradients.
            optimizer_vae_generator.zero_grad() # Clear gradients.
            running_loss += loss 
            running_loss_dis += loss_discriminator 
            data.batch=None
            data.edge_index=data.edge_index.cpu()
            data.x=data.x.cpu()
            data.y=data.y.cpu()
            pred = out.argmax(dim=1)
        print("final_loss")
        running_loss /= len(train_dataset_malware)
        running_loss_dis /= len(train_dataset_malware)
        vgae_loss=running_loss-running_loss_dis
        print("loss of vgae",vgae_loss)
        print("loss of generator",running_loss)
        print('loss of discriminator',running_loss_dis)

current iteration 0
malware generation complete
graph generation 0
[[   0    0]
 [ 214 8262]]
true positive rate after generation 0.9747522416234072
[[   0    0]
 [ 214 8262]]
2586
0
training benign discriminator
Epoch: 000, discriminator Train Acc: 1.0000,discriminator Train loss:0.0000
648
7826
training malware discriminator
Epoch: 000, discriminator Train Acc: 0.9024,discriminator Train loss:74.9137
0
8474
training malware discriminator
Epoch: 001, discriminator Train Acc: 0.9749,discriminator Train loss:0.1228
0
8474
training malware discriminator
Epoch: 002, discriminator Train Acc: 0.9749,discriminator Train loss:0.1213
0
8474
training malware discriminator
Epoch: 003, discriminator Train Acc: 0.9749,discriminator Train loss:0.1203
0
8474
training malware discriminator
Epoch: 004, discriminator Train Acc: 0.9749,discriminator Train loss:0.1179
0
8474
training malware discriminator
Epoch: 005, discriminator Train Acc: 0.9749,discriminator Train loss:0.1182
0
8474
training malware 

final_loss
loss of vgae tensor(0.2030, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(1.2394, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(1.0363, device='cuda:0', grad_fn=<DivBackward0>)
current iteration 4
malware generation complete
graph generation 0
[[   0    0]
 [5658 2818]]
true positive rate after generation 0.3324681453515809
[[   0    0]
 [5658 2818]]
2575
11
training benign discriminator
Epoch: 004, discriminator Train Acc: 0.9957,discriminator Train loss:0.0528
7205
1269
training malware discriminator
Epoch: 000, discriminator Train Acc: 0.6217,discriminator Train loss:1742.6110
6279
2195
training malware discriminator
Epoch: 001, discriminator Train Acc: 0.5797,discriminator Train loss:666.8544
6231
2243
training malware discriminator
Epoch: 002, discriminator Train Acc: 0.5827,discriminator Train loss:502.4936
6181
2293
training malware discriminator
Epoch: 003, discriminator Train Acc: 0.5787,discriminator Train loss:397.7277
6

final_loss
loss of vgae tensor(0.2035, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(0.2035, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(0., device='cuda:0', grad_fn=<DivBackward0>)
final_loss
loss of vgae tensor(0.2028, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(0.2028, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(0., device='cuda:0', grad_fn=<DivBackward0>)
final_loss
loss of vgae tensor(0.2022, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(0.2022, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(0., device='cuda:0', grad_fn=<DivBackward0>)
current iteration 8
malware generation complete
graph generation 0
[[   0    0]
 [5374 3102]]
true positive rate after generation 0.36597451628126476
[[   0    0]
 [5374 3102]]
2586
0
training benign discriminator
Epoch: 008, discriminator Train Acc: 1.0000,discriminator Train loss:0.0000
8474
0
training malware di

8474
0
training malware discriminator
Epoch: 006, discriminator Train Acc: 0.4890,discriminator Train loss:6851.5324
8474
0
training malware discriminator
Epoch: 007, discriminator Train Acc: 0.4890,discriminator Train loss:6070.7560
8474
0
training malware discriminator
Epoch: 008, discriminator Train Acc: 0.4890,discriminator Train loss:5375.1708
8474
0
training malware discriminator
Epoch: 009, discriminator Train Acc: 0.4890,discriminator Train loss:4743.2946
final_loss
loss of vgae tensor(0.2040, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(0.2040, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(0., device='cuda:0', grad_fn=<DivBackward0>)
final_loss
loss of vgae tensor(0.2036, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(0.2036, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(0., device='cuda:0', grad_fn=<DivBackward0>)
final_loss
loss of vgae tensor(0.2045, device='cuda:0', grad_fn=<SubBackward

6673
1801
training malware discriminator
Epoch: 003, discriminator Train Acc: 0.6172,discriminator Train loss:903.0884
6587
1887
training malware discriminator
Epoch: 004, discriminator Train Acc: 0.6101,discriminator Train loss:841.7413
6530
1944
training malware discriminator
Epoch: 005, discriminator Train Acc: 0.6046,discriminator Train loss:781.2756
6477
1997
training malware discriminator
Epoch: 006, discriminator Train Acc: 0.6037,discriminator Train loss:718.1527
6517
1957
training malware discriminator
Epoch: 007, discriminator Train Acc: 0.6042,discriminator Train loss:650.6868
6451
2023
training malware discriminator
Epoch: 008, discriminator Train Acc: 0.6007,discriminator Train loss:607.9024
6509
1965
training malware discriminator
Epoch: 009, discriminator Train Acc: 0.5990,discriminator Train loss:552.6552
final_loss
loss of vgae tensor(0.2022, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(217.3729, device='cuda:0', grad_fn=<DivBackward0>)
loss of dis

...

final_loss
loss of vgae tensor(0.2039, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(0.2039, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(0., device='cuda:0', grad_fn=<DivBackward0>)
current iteration 84
malware generation complete
graph generation 0
[[   0    0]
 [5611 2865]]
true positive rate after generation 0.3380132137800849
[[   0    0]
 [5611 2865]]
2586
0
training benign discriminator
Epoch: 084, discriminator Train Acc: 1.0000,discriminator Train loss:0.0000
8474
0
training malware discriminator
Epoch: 000, discriminator Train Acc: 0.6620,discriminator Train loss:4090.0217
8405
69
training malware discriminator
Epoch: 001, discriminator Train Acc: 0.6591,discriminator Train loss:3283.2452
7592
882
training malware discriminator
Epoch: 002, discriminator Train Acc: 0.6264,discriminator Train loss:2651.6094
6831
1643
training malware discriminator
Epoch: 003, discriminator Train Acc: 0.5994,discriminator Train loss:2415.1615
6571
190

final_loss
loss of vgae tensor(0.2037, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(0.2037, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(0., device='cuda:0', grad_fn=<DivBackward0>)
final_loss
loss of vgae tensor(0.2026, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(0.2026, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(0., device='cuda:0', grad_fn=<DivBackward0>)
final_loss
loss of vgae tensor(0.2037, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(0.2037, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(0., device='cuda:0', grad_fn=<DivBackward0>)
current iteration 88
malware generation complete
graph generation 0
[[   0    0]
 [4522 3954]]
true positive rate after generation 0.46649362907031616
[[   0    0]
 [4522 3954]]
2586
0
training benign discriminator
Epoch: 088, discriminator Train Acc: 1.0000,discriminator Train loss:0.0000
8474
0
training malware d

8474
0
training malware discriminator
Epoch: 006, discriminator Train Acc: 0.8867,discriminator Train loss:4955.4297
8474
0
training malware discriminator
Epoch: 007, discriminator Train Acc: 0.8867,discriminator Train loss:4906.4240
8474
0
training malware discriminator
Epoch: 008, discriminator Train Acc: 0.8867,discriminator Train loss:4851.9160
8474
0
training malware discriminator
Epoch: 009, discriminator Train Acc: 0.8867,discriminator Train loss:4789.3507
final_loss
loss of vgae tensor(0.2023, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(0.2023, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(0., device='cuda:0', grad_fn=<DivBackward0>)
final_loss
loss of vgae tensor(0.2041, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(0.2041, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(0., device='cuda:0', grad_fn=<DivBackward0>)
final_loss
loss of vgae tensor(0.2030, device='cuda:0', grad_fn=<SubBackward

8474
0
training malware discriminator
Epoch: 003, discriminator Train Acc: 0.6321,discriminator Train loss:9103.3701
8474
0
training malware discriminator
Epoch: 004, discriminator Train Acc: 0.6321,discriminator Train loss:8637.8004
8474
0
training malware discriminator
Epoch: 005, discriminator Train Acc: 0.6321,discriminator Train loss:8162.7339
8474
0
training malware discriminator
Epoch: 006, discriminator Train Acc: 0.6321,discriminator Train loss:7644.6381
8474
0
training malware discriminator
Epoch: 007, discriminator Train Acc: 0.6321,discriminator Train loss:7105.4232
8474
0
training malware discriminator
Epoch: 008, discriminator Train Acc: 0.6321,discriminator Train loss:6531.7655
8474
0
training malware discriminator
Epoch: 009, discriminator Train Acc: 0.6321,discriminator Train loss:5949.7707
final_loss
loss of vgae tensor(0.2030, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(0.2030, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tenso

2571
15
training benign discriminator
Epoch: 099, discriminator Train Acc: 0.9942,discriminator Train loss:11.1512
8474
0
training malware discriminator
Epoch: 000, discriminator Train Acc: 0.7164,discriminator Train loss:12166.1729
8474
0
training malware discriminator
Epoch: 001, discriminator Train Acc: 0.7164,discriminator Train loss:11867.0075
8474
0
training malware discriminator
Epoch: 002, discriminator Train Acc: 0.7164,discriminator Train loss:11557.3140
8474
0
training malware discriminator
Epoch: 003, discriminator Train Acc: 0.7164,discriminator Train loss:11268.4621
8474
0
training malware discriminator
Epoch: 004, discriminator Train Acc: 0.7164,discriminator Train loss:10924.2611
8474
0
training malware discriminator
Epoch: 005, discriminator Train Acc: 0.7164,discriminator Train loss:10575.1380
8474
0
training malware discriminator
Epoch: 006, discriminator Train Acc: 0.7164,discriminator Train loss:10226.2973
8474
0
training malware discriminator
Epoch: 007, discrimin